In [1]:
import math

import duckdb
import plotly.express as px
import plotly.graph_objects as go

from utils import wrap_to_string, trimmer

conn = duckdb.connect("dataset.db")


## 2.1 Duplicates

In [7]:
# Check number of duplicates while ignoring the index feature
query = """
SELECT column_name
FROM information_schema.columns
"""
cols = conn.sql(query).df()
cols = cols[cols["column_name"]!="Accident_Index"]
all_cols = wrap_to_string(cols)

In [8]:
all_cols

'"Vehicle_Reference_df_res" ,"Vehicle_Type" ,"Towing_and_Articulation" ,"Vehicle_Manoeuvre" ,"Vehicle_Location-Restricted_Lane" ,"Junction_Location" ,"Skidding_and_Overturning" ,"Hit_Object_in_Carriageway" ,"Vehicle_Leaving_Carriageway" ,"Hit_Object_off_Carriageway" ,"1st_Point_of_Impact" ,"Was_Vehicle_Left_Hand_Drive?" ,"Journey_Purpose_of_Driver" ,"Sex_of_Driver" ,"Age_of_Driver" ,"Age_Band_of_Driver" ,"Engine_Capacity_(CC)" ,"Propulsion_Code" ,"Age_of_Vehicle" ,"Driver_Home_Area_Type" ,"Location_Easting_OSGR" ,"Location_Northing_OSGR" ,"Longitude" ,"Latitude" ,"Police_Force" ,"Accident_Severity" ,"Number_of_Vehicles" ,"Number_of_Casualties" ,"Date" ,"Day_of_Week" ,"Time" ,"Local_Authority_(District)" ,"Local_Authority_(Highway)" ,"1st_Road_Class" ,"1st_Road_Number" ,"Road_Type" ,"Speed_limit" ,"Junction_Detail" ,"Junction_Control" ,"2nd_Road_Class" ,"2nd_Road_Number" ,"Pedestrian_Crossing-Human_Control" ,"Pedestrian_Crossing-Physical_Facilities" ,"Light_Conditions" ,"Weather_Conditi

In [10]:
query = f"""
SELECT {all_cols},COUNT(*) 
FROM dataset
GROUP BY {all_cols}
HAVING COUNT(*)>1
"""
conn.sql(query).show()

┌──────────────────────┬──────────────┬───┬──────────────────────┬─────────────────────┬──────────────┐
│ Vehicle_Reference_…  │ Vehicle_Type │ … │ Casualty_Home_Area…  │ Casualty_IMD_Decile │ count_star() │
│        int64         │    double    │   │        double        │       double        │    int64     │
├──────────────────────┼──────────────┼───┼──────────────────────┼─────────────────────┼──────────────┤
│                    1 │          1.0 │ … │                  1.0 │                 9.0 │            2 │
│                    2 │          2.0 │ … │                  1.0 │                 4.0 │            2 │
│                    2 │          1.0 │ … │                 NULL │                NULL │            2 │
│                    2 │          1.0 │ … │                  1.0 │                 5.0 │            2 │
│                    2 │          1.0 │ … │                  1.0 │                 3.0 │            2 │
│                    2 │          1.0 │ … │                  1.0